## Homework 04
  
The [Conversation AI](https://conversationai.github.io/) team, a research initiative founded by [Jigsaw](https://jigsaw.google.com/) and Google (both a part of Alphabet) are working on tools to help improve online conversation. One area of focus is the study of negative online behaviors, like toxic comments (i.e. comments that are rude, disrespectful or otherwise likely to make someone leave a discussion).   
  
Kaggle are currently hosting their [second competition](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge#description) on this research. The challenge is to create a model that is capable of detecting different types of of toxicity like threats, obscenity, insults, and identity-based hate better than Perspective’s current models. The competitions use a dataset of comments from Wikipedia’s talk page edits. Improvements to the current model will hopefully help online discussion become more productive and respectful.

We shall be using this dataset to benchmark a number of ML models. While the focus of the current competition is to mitigate bias, we will not be using the metric used in the competition. Instead we will be focusing on a simpler metric [Area under the Curve (or AUC)](https://www.kaggle.com/learn-forum/53782) which is suitable to unbalanced binary datasets. Also, we shall not consider different levels of Toxicity; we shall purely take anything marked over the 0.5 level in the measured toxicity range as toxic, and anything underneath as non-toxic. 

We have created a jupyter notbook with some of the tools to model this problem in Deep Learning, using Logistic regression and MLP. Your challenge will be to fill in the models and benchmark the AUC you achieve on these models.

We shall be using the keras deep learning package. As you may know, this is an API into DL frameworks, but is most commonly backed by Tensorflow. [keras.io](keras.io) is a great source for documentation and examples on layers available andn functionality. 

**Have fun!!**


*Disclaimer: the dataset used contains text that may be considered profane, vulgar, or offensive.*

### Set up packages

In [1]:
import os
import tensorflow as tf
import keras
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

### Load data
Let's load our data and take a peak.   
The competition metrics and fields have too much detail too cover here.   
We will just focus on the comment of the users and whether it was deemed toxic (target>0.5) or not.    

In [2]:
# Lets look at what files we have available. 
os.listdir()

['w251_homework04.ipynb', '.ipynb_checkpoints', 'train.csv.zip']

In [3]:
train_df = pd.read_csv('train.csv.zip', 
                       dtype={'comment_text':str},
                       usecols=['comment_text', 'target'],
                       compression = 'zip')
train_df['target'] = (train_df['target']>0.5).astype(int)
print("Dataframe Number of (rows, columns) : {}".format(train_df.shape))

Dataframe Number of (rows, columns) : (1804874, 2)


In [4]:
pd.options.display.max_colwidth = 200
train_df.head(10)

,target,comment_text
0,0,"This is so cool. It's like, 'would you want your mother to read this??' Really great idea, well done!"
1,0,"Thank you!! This would make my life a lot less anxiety-inducing. Keep it up, and don't let anyone get in your way!"
2,0,This is such an urgent design problem; kudos to you for taking it on. Very impressive!
3,0,Is this something I'll be able to install on my site? When will you be releasing it?
4,1,haha you guys are a bunch of losers.
5,1,ur a sh*tty comment.
6,0,hahahahahahahahhha suck it.
7,0,FFFFUUUUUUUUUUUUUUU
8,0,The ranchers seem motivated by mostly by greed; no one should have the right to allow their animals destroy public land.
9,0,It was a great show. Not a combo I'd of expected to be good together but it was.


### Create validation data set
Lets randomly 66/33 split the data into a training and validation set.   
**No change needed here - note, please do not change the KFold split parameters, keeping it consistent will help us debug.**

In [5]:
cv = KFold(n_splits=3, shuffle=True, random_state=42)
trn_ids, val_ids = next(cv.split(train_df))
x_train, x_valid = train_df['comment_text'][trn_ids], train_df['comment_text'][val_ids]
y_train, y_valid = train_df['target'].values[trn_ids], train_df['target'].values[val_ids]

### Vectorize Count of tokens

![Count Vectorizer](https://cdn-images-1.medium.com/max/1600/1*LD5XaHzOnoniU4p137FL5g.jpeg)  
We shall start off performing some CPU based Deep Learning operations. Sparse matrices are better run on CPU.    
Do not underestimate CPU based Deep Learning such as MLP; these models can be very powerful and outperform complex much more complex DL models.   
Here we create a sparse matrix from the text with 200K of the most common unigram and bigrams.  
**Your task here is to convert the collection of text documents (found in the `comment_text` field) to a matrix of token counts.  
This can be done using the [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) in scikit_learn.  
After creating the vectorizer, fit it based on the train matrix `x_train` and use this vectorizer to transform both the `x_train` and `x_valid` sets.   
Create sparse matrices called `X_trn_mat` and `X_val_mat`, and please call your vectorizer: `vectorizer`.  
Use the parameters max features = 200000 and the token pattern `\w+`. This token pattern matches one or more word characters (same as `[a-zA-Z0-9_]`) only. All other characters are stripped.
Also, we would like to count both unigrams and bigrams (pairs of words), so set the ngram range to `(1,2)`**

In [6]:
#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
# Create a CountVectorizer, called `vectorizer`
# And create sparse matrices X_trn_mat & X_val_mat
#<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

vectorizer = CountVectorizer(max_features = 200000,
                             token_pattern="\w+",
                             ngram_range = (1,2))

X_trn_mat = vectorizer.fit_transform(x_train)
print("Completed fitting of training set")

X_val_mat = vectorizer.transform(x_valid)
print("Completed mapping of validation set")


Completed fitting of training set
Completed mapping of validation set


In [7]:
print(vectorizer.get_feature_names()[:10])
print(vectorizer.get_feature_names()[100000:100000+10])

['0', '0 0', '0 00', '0 01', '0 05', '0 1', '0 2', '0 25', '0 3', '0 4']
['make more', 'make most', 'make much', 'make my', 'make new', 'make no', 'make noise', 'make obama', 'make of', 'make on']


In [8]:
X_trn_mat

<1203249x200000 sparse matrix of type '<class 'numpy.int64'>'
	with 89432534 stored elements in Compressed Sparse Row format>

### Logistic Regression

![Logistic Regression](https://upload.wikimedia.org/wikipedia/commons/6/6d/Exam_pass_logistic_curve.jpeg)
  
Lets start off with a simple Logistic Regression, which is the very basic [sigmoid activation function](https://en.wikipedia.org/wiki/Sigmoid_function) used in DL.  
Notice we have no hidden layers, we take as input the whole sparse matrix, and as output the binary classifier prediction (`0<=output<=1`).  
The model has 200001 parameters. One coefficient per column in the sparse matrx, plus one bias variable - each of which is learned using gradient descent. 


In [9]:
model_in = keras.Input(shape=(X_trn_mat.shape[1],), dtype='float32', sparse=True)
out = keras.layers.Dense(1, activation='sigmoid')(model_in)
model = keras.Model(inputs=model_in, outputs=out)
model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(lr=1e-2))
model.summary()

W0602 02:30:20.642560 547954618384 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0602 02:30:20.650950 547954618384 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:539: The name tf.sparse_placeholder is deprecated. Please use tf.compat.v1.sparse_placeholder instead.

W0602 02:30:20.752102 547954618384 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0602 02:30:20.868355 547954618384 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:1133: The name tf.sparse_tensor_dense_matmul is deprecated. Please use tf.sparse.sparse_dense_matmul instead.

W0602 02:30:21.044852 547

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 200000)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 200001    
Total params: 200,001
Trainable params: 200,001
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.fit(X_trn_mat, y_train, epochs=2, batch_size=2**13, validation_data=(X_val_mat, y_valid))

Train on 1203249 samples, validate on 601625 samples
Epoch 1/2
1203249/1203249 [==============================] - 64s 53us/step - loss: 0.2402 - val_loss: 0.1770
Epoch 2/2
1203249/1203249 [==============================] - 39s 32us/step - loss: 0.1324 - val_loss: 0.1488


In [11]:
preds_lr = model.predict(X_val_mat).flatten()
print('AUC score : {:.5f}'.format(roc_auc_score(y_valid, preds_lr)))

AUC score : 0.88549


Look at the coefficients to see which words are driving toxic and non-toxic sentences. 

In [12]:
feats = np.array(vectorizer.get_feature_names())
importance_index = model.get_weights()[0].flatten().argsort()
print('Top toxic tokens : \n{}'.format(feats[importance_index[-10:]].tolist()))
print('\nTop non-toxic tokens : \n{}'.format(feats[importance_index[:10]].tolist()))

Top toxic tokens : 
['shit', 'damn', 'pathetic', 'stupid', 'morons', 'hypocrite', 'idiot', 'stupidity', 'idiots', 'idiotic']

Top non-toxic tokens : 
['well said', 'amen', 'propaganda from', 'oh brother', 'bingo', 'great comment', 'the assertion', 'some not', 'panhandling', 'awesome']


### MLP

![MLP](https://www.researchgate.net/profile/Mouhammd_Alkasassbeh/publication/309592737/figure/fig2/AS:423712664100865@1478032379613/MultiLayer-Perceptron-MLP-sturcture-334-MultiLayer-Perceptron-Classifier-MultiLayer.png)

Here we shall create a Multi-layer perceptron. Although relatively simple, these can be very powerful models and also suited to low compute power. 
**Please add three hidden layers to the network using a `relu` activation function.  
You can refer to this [script](https://www.kaggle.com/lopuhin/mercari-golf-0-3875-cv-in-75-loc-1900-s). This script contains an MLP which took first place in the *Mercari Price Suggestion Challenge*.   
Note, you can do this task by only adding four lines of code. You should see a large increase in AUC over the Logistic Regression.**  
Never underestimate the power of an MLP!!

In [13]:
model_in = keras.Input(shape=(X_trn_mat.shape[1],), dtype='float32', sparse=True)
#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
# Please fill in the next lines with the three hidden layers and the output layer. 
#<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

out = keras.layers.Dense(192, activation='relu')(model_in)
out = keras.layers.Dense(64, activation='relu')(out)
out = keras.layers.Dense(64, activation='relu')(out)
out = keras.layers.Dense(1)(out)
model = keras.Model(inputs=model_in, outputs=out)
model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(lr=1e-3))
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 200000)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 192)               38400192  
_________________________________________________________________
dense_3 (Dense)              (None, 64)                12352     
_________________________________________________________________
dense_4 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 65        
Total params: 38,416,769
Trainable params: 38,416,769
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.fit(X_trn_mat, y_train, batch_size=2**13, epochs=2, verbose=1, validation_data=(X_val_mat, y_valid))
preds_mlp = model.predict(X_val_mat).flatten()

Train on 1203249 samples, validate on 601625 samples
Epoch 1/2
1203249/1203249 [==============================] - 128s 107us/step - loss: 0.1719 - val_loss: 0.1471
Epoch 2/2
1203249/1203249 [==============================] - 113s 94us/step - loss: 0.0932 - val_loss: 0.1760


In [15]:
print('AUC score : {:.5f}'.format(roc_auc_score(y_valid, preds_mlp)))

AUC score : 0.89525


### MLP with regularisation

Now lets try regularization.  
**Copy the above MLP model and create a new one adding regularization into the MLP hidden layers.  
    Add l2 regularisation to each of the dense hidden layers. Check on [keras.io](https://keras.io) to find details on how to add l2 regularization. Play are around with different level of regularization to see when you achieve optimal results.   
Generally it is good to choose parameters like regularization by moving up and down in factors of `10`.  
Can you improve on your previous AUC results by using reglarisation?**

In [16]:
from keras.regularizers import l2

model_in = keras.Input(shape=(X_trn_mat.shape[1],), dtype='float32', sparse=True)
#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
# Please fill in the next lines with the three hidden layers and the output layer. 
#<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
l2coeff = 0.01
out = keras.layers.Dense(192, activation='relu', kernel_regularizer=l2(l2coeff))(model_in)
out = keras.layers.Dense(64, activation='relu', kernel_regularizer=l2(l2coeff))(out)
out = keras.layers.Dense(64, activation='relu', kernel_regularizer=l2(l2coeff))(out)
out = keras.layers.Dense(1)(out)
model = keras.Model(inputs=model_in, outputs=out)
model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(lr=1e-3))
model.summary()
print("Fitting...")
model.fit(X_trn_mat, y_train, batch_size=2**13, epochs=2, verbose=1, validation_data=(X_val_mat, y_valid))
print("Inferring...")
preds_mlp = model.predict(X_val_mat).flatten()
print("Checking...")
print('AUC score : {:.5f}'.format(roc_auc_score(y_valid, preds_mlp)))


Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 200000)            0         
_________________________________________________________________
dense_6 (Dense)              (None, 192)               38400192  
_________________________________________________________________
dense_7 (Dense)              (None, 64)                12352     
_________________________________________________________________
dense_8 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 65        
Total params: 38,416,769
Trainable params: 38,416,769
Non-trainable params: 0
_________________________________________________________________
Fitting...
Train on 1203249 samples, validate on 601625 samples
Epoch 1/2
1203249/1203249 [======================

In [17]:
from keras.regularizers import l2

model_in = keras.Input(shape=(X_trn_mat.shape[1],), dtype='float32', sparse=True)
#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
# Please fill in the next lines with the three hidden layers and the output layer. 
#<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
l2coeff = 0.001
out = keras.layers.Dense(192, activation='relu', kernel_regularizer=l2(l2coeff))(model_in)
out = keras.layers.Dense(64, activation='relu', kernel_regularizer=l2(l2coeff))(out)
out = keras.layers.Dense(64, activation='relu', kernel_regularizer=l2(l2coeff))(out)
out = keras.layers.Dense(1)(out)
model = keras.Model(inputs=model_in, outputs=out)
model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(lr=1e-3))
model.summary()
print("Fitting...")
model.fit(X_trn_mat, y_train, batch_size=2**13, epochs=2, verbose=1, validation_data=(X_val_mat, y_valid))
print("Inferring...")
preds_mlp = model.predict(X_val_mat).flatten()
print("Checking...")
print('AUC score : {:.5f}'.format(roc_auc_score(y_valid, preds_mlp)))


Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 200000)            0         
_________________________________________________________________
dense_10 (Dense)             (None, 192)               38400192  
_________________________________________________________________
dense_11 (Dense)             (None, 64)                12352     
_________________________________________________________________
dense_12 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 65        
Total params: 38,416,769
Trainable params: 38,416,769
Non-trainable params: 0
_________________________________________________________________
Fitting...
Train on 1203249 samples, validate on 601625 samples
Epoch 1/2
1203249/1203249 [======================

In [18]:
from keras.regularizers import l2

model_in = keras.Input(shape=(X_trn_mat.shape[1],), dtype='float32', sparse=True)
#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
# Please fill in the next lines with the three hidden layers and the output layer. 
#<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
l2coeff = 0.0001
out = keras.layers.Dense(192, activation='relu', kernel_regularizer=l2(l2coeff))(model_in)
out = keras.layers.Dense(64, activation='relu', kernel_regularizer=l2(l2coeff))(out)
out = keras.layers.Dense(64, activation='relu', kernel_regularizer=l2(l2coeff))(out)
out = keras.layers.Dense(1)(out)
model = keras.Model(inputs=model_in, outputs=out)
model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(lr=1e-3))
model.summary()
print("Fitting...")
model.fit(X_trn_mat, y_train, batch_size=2**13, epochs=2, verbose=1, validation_data=(X_val_mat, y_valid))
print("Inferring...")
preds_mlp = model.predict(X_val_mat).flatten()
print("Checking...")
print('AUC score : {:.5f}'.format(roc_auc_score(y_valid, preds_mlp)))


Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 200000)            0         
_________________________________________________________________
dense_14 (Dense)             (None, 192)               38400192  
_________________________________________________________________
dense_15 (Dense)             (None, 64)                12352     
_________________________________________________________________
dense_16 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 65        
Total params: 38,416,769
Trainable params: 38,416,769
Non-trainable params: 0
_________________________________________________________________
Fitting...
Train on 1203249 samples, validate on 601625 samples
Epoch 1/2
1203249/1203249 [======================